In [ ]:
import numpy as np 
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

In [ ]:
train_data["Relatives"] = train_data["SibSp"] + train_data["Parch"]
train_data.loc[train_data["Relatives"] > 0, "Alone"] = 'No'
train_data.loc[train_data["Relatives"] == 0, "Alone"] = 'Yes'

In [ ]:
train_data.info()

In [ ]:
#train_data["Age"].fillna(train_data["Age"].median(), inplace = True)
train_data["Title"] = train_data["Name"].map(lambda name:name.split(',')[1].split('.')[0].strip())
train_data['Embarked'] = train_data['Embarked'].fillna(train_data['Embarked'].mode())

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_data["Sex"]= le.fit_transform(train_data["Sex"])
train_data["Embarked"]= le.fit_transform(train_data["Embarked"])
train_data["Title"]= le.fit_transform(train_data["Title"])

In [ ]:
train_data.info()

In [ ]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

In [ ]:
mean = train_data["Age"].mean()
std = test_data["Age"].std()
is_null = train_data["Age"].isnull().sum()
rand_age = np.random.randint(mean - std, mean + std, size = is_null)
age_slice = train_data["Age"].copy()
age_slice[np.isnan(age_slice)] = rand_age
train_data["Age"] = age_slice
train_data["Age"] = train_data["Age"].astype(int)

In [ ]:
mean = train_data["Age"].mean()
std = test_data["Age"].std()
is_null = test_data["Age"].isnull().sum()
rand_age = np.random.randint(mean - std, mean + std, size = is_null)
age_slice = test_data["Age"].copy()
age_slice[np.isnan(age_slice)] = rand_age
test_data["Age"] = age_slice
test_data["Age"] = test_data["Age"].astype(int)

In [ ]:
test_data["Relatives"] = test_data["SibSp"] + test_data["Parch"]
test_data.loc[test_data["Relatives"] > 0, "Alone"] = "No"
test_data.loc[test_data["Relatives"] == 0, "Alone"] = "Yes"

In [ ]:
#test_data["Age"].fillna(test_data["Age"].median(), inplace = True)
test_data["Title"] = test_data["Name"].map(lambda name:name.split(',')[1].split('.')[0].strip())
test_data["Fare"].fillna(test_data["Fare"].median(), inplace = True)

In [ ]:
test_data["Sex"]= le.fit_transform(test_data["Sex"])
test_data["Embarked"]= le.fit_transform(test_data["Embarked"])
test_data["Title"]= le.fit_transform(test_data["Title"])
#test_data['travelled_alone']= le.fit_transform(test_data['travelled_alone'])

In [ ]:
test_data.info()

In [ ]:
test_data.info()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

Y_train = train_data["Survived"]

features = ["Pclass", "Sex", "Relatives", "Alone", "SibSp", "Parch", "Title", "Fare", "Embarked"]
#features = ["Pclass", "Sex", "relatives", "Title", "Embarked"]
X_train = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])
#'''
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)
#'''
'''
parameter_grid = {"max_depth" : [4, 6, 8, 10, 12],
                  "criterion": ["gini", "entropy"],
                  "n_estimators": [100, 50, 10, 5],
                  "max_features": ["sqrt", "auto", "log2"],
                  "random_state": [0, 1],
                  "bootstrap": [True, False]}
scoring = {'AUC': 'roc_auc', 'Accuracy': make_scorer(accuracy_score)}
model = GridSearchCV(RandomForestClassifier(), scoring=scoring, param_grid=parameter_grid, cv=5, verbose=1)
'''
#model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model = LogisticRegression()
model.fit(X_train, Y_train)
predictions = model.predict(X_test)
print(model.score(X_train, Y_train))
      

output = pd.DataFrame({"PassengerId": test_data.PassengerId, "Survived": predictions})
output.to_csv("submission.csv", index=False)
print("Your submission was successfully saved!")